# Setup

In [ ]:
!conda env create -n ICT3104 --file environment.yml

In [ ]:
!conda env list

In [ ]:
!conda activate ICT3104

In [ ]:
# Ensure environment is working
import torch
import timm
import sklearn
import numpy

# Example Use

# Apply Module